# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import pandas as pd

if '../../../notebooks/' not in sys.path:
    sys.path.append('../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from model_trainer import BaseModel, Optimizer

# Start Experiment Runner

In [ ]:
req = cts.ICAO_REQ.MOUTH
ds = GTName.FVC
aligned = True
aligned_str = 'aligned' if aligned else 'not_aligned'
model_name = f'{ds.value.lower()}_{aligned_str}/{req.value.lower()}'

df = pd.read_csv('analysis/single_task_exps/processed_experiments_data.csv')
model_exp_id = df[(df.property_icao_reqs.str.contains(req.value.lower())) & \
                   (df.property_aligned == aligned) & \
                  (df.property_gt_names.str.contains(ds.value.lower()))].id.values[0]

kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'train_vgg16',
        'description': f'Training network for {req.value.upper()} requisite',
        'tags': ['vgg16', 'ground truths', 'adamax', ds.value.lower(), 'binary_output', req.value.lower(),
                 'single_task', f'{model_exp_id}'],
        'src_files': ['exp_runner.py', 'data_processor.py', 'model_trainer.py', 'model_evaluator.py']
    },
    'properties': {
#         'tagger_model': OpenfaceMouth(),
#         'dl_names': {'train_validation':
#                            [DLName.VGGFACE2, DLName.CALTECH, DLName.FVC_PYBOSSA, 
#                             DLName.CVL, DLName.FEI_DB, DLName.GEORGIA_TECH, DLName.COLOR_FERET,
#                             DLName.ICPR04, DLName.IMFDB, DLName.IJBC, DLName.LFW, DLName.CASIA_WF,
#                             DLName.GENKI4K_DB],
#                       'test': [],
        #'reqs': list(cts.ICAO_REQ),
        'reqs': [req],
        'aligned': aligned,
        'use_gt_data': True,
        'gt_names': {
            'train_validation': [],
            'test': [],
            'train_validation_test': [ds]
        },
        'balance_input_data': False,
        'train_model': False,
        'save_trained_model': False,
        'model_name': f'single_task/{model_name}',
        'orig_model_experiment_id': model_exp_id,
        'sample_training_data': False,
        'sample_prop': 1.
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 64,
        'n_epochs': 10,
        'early_stopping': 10,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'train_prop': 0.9,
        'validation_prop': 0.05,
        'test_prop': 0.05,
        'validation_split': 0.1,
        'dropout': 0.3
    }
}

runner = ExperimentRunner(**kwargs)

# Load Data

In [ ]:
runner.load_training_data()

# Sampling Training Data

In [ ]:
runner.sample_training_data()

# Data Balancing

In [ ]:
runner.balance_input_data()

# Data Generators

In [ ]:
runner.setup_data_generators()

# Use Neptune

In [ ]:
runner.start_neptune()
runner.create_experiment()

# Labels Distribution

In [ ]:
runner.summary_labels_dist()

In [ ]:
runner.summary_gen_labels_dist()

# Create Model

In [ ]:
runner.create_model()

# Vizualize Model

In [ ]:
runner.vizualize_model(outfile_path=f"figs/{model_name.replace('/','_')}.png")

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

In [ ]:
runner.test_model()

# Vizualize Model Classification

In [ ]:
runner.vizualize_predictions(n_imgs=50)

## False Positives

In [ ]:
runner.vizualize_predictions(n_imgs=50, show_only_fp=True)

## False Negatives

In [ ]:
runner.vizualize_predictions(n_imgs=50, show_only_fn=True)

## True Positives

In [ ]:
runner.vizualize_predictions(n_imgs=50, show_only_tp=True)

## True Negatives

In [ ]:
runner.vizualize_predictions(n_imgs=50, show_only_tn=True)

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()